In [1]:
# =============================================================================
# PART 1: SETUP, DATA LOADING & CUSTOM TRANSFORMERS (MODIFIED)
# -----------------------------------------------------------------------------
# This section contains all necessary imports and loads the initial dataset.
# Most importantly, it defines the powerful, custom scikit-learn transformers
# from your original code. The MissingFlagImputer has been made more robust.
# =============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from category_encoders import TargetEncoder
from lightgbm import LGBMRegressor
import warnings
import shap

# --- Setup ---
# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')
# Set a consistent plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (18, 8)
# Set a random seed for reproducibility
RANDOM_STATE = 42

# --- Data Loading ---
print("PART 1: Loading and preparing data...")
try:
    # This uses your real dataset. Ensure the path is correct.
    df = pd.read_csv('/Users/shayan/Desktop/IDS2/Stattkueche/df_weather2.csv', parse_dates=['DateOfService', 'DateOfOrder', 'DateOfCancel'])
    print("Successfully loaded df_weather2.csv.")
except FileNotFoundError:
    print("FATAL ERROR: df_weather2.csv not found. Please update the file path.")
    exit()

# --- Initial Feature Creation ---
# This is the target variable for the final regression model.
df['net_qty'] = df['OrderQty'] - df['CanceledQty']
print("Initial DataFrame shape:", df.shape)


# --- Custom Transformer Definitions (from your original code) ---
# We define all your custom classes here to be used later in the pipeline.

class HistCancelRateTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, group_keys=('Site','MenuBase'), value_col='net_qty', out_col='hist_cancel_rate'):
        self.group_keys = group_keys
        self.value_col  = value_col
        self.out_col    = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        self.hist_    = X.groupby(keys)[self.value_col].mean()
        self.default_ = self.hist_.median()
        return self
    def transform(self, X):
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X[keys].values]
        X_copy = X.copy()
        X_copy[self.out_col] = [self.hist_.get(t, self.default_) for t in tuples]
        return X_copy

class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, profile_feats, group_keys=('Site','MenuBase'),
                 n_clusters=5, out_col='cluster_id'):
        self.profile_feats = profile_feats
        self.group_keys    = group_keys
        self.n_clusters    = n_clusters
        self.out_col       = out_col
    def fit(self, X, y=None):
        keys = list(self.group_keys)
        prof = (X.groupby(keys)[self.profile_feats].mean().reset_index())
        prof[self.profile_feats] = prof[self.profile_feats].fillna(prof[self.profile_feats].median())
        self.scaler_ = StandardScaler().fit(prof[self.profile_feats])
        scaled      = self.scaler_.transform(prof[self.profile_feats])
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=RANDOM_STATE).fit(scaled)
        tuples      = [tuple(r) for r in prof[keys].values]
        self.cluster_map_ = dict(zip(tuples, self.kmeans_.labels_))
        self.default_     = int(np.median(self.kmeans_.labels_))
        return self
    def transform(self, X):
        X_copy = X.copy()
        keys   = list(self.group_keys)
        tuples = [tuple(r) for r in X_copy[keys].values]
        X_copy[self.out_col] = [self.cluster_map_.get(t, self.default_) for t in tuples]
        return X_copy

class InCVTargetEncoder(BaseEstimator, TransformerMixin):
    """
    A target encoder that calculates encodings within cross-validation folds
    to prevent data leakage.
    """
    def __init__(self, cols, smoothing=1.0):
        self.cols = cols
        self.smoothing = smoothing
    def fit(self, X, y):
        self.global_mean_ = y.mean()
        self.mapping_ = {}
        for c in self.cols:
            df_temp = pd.DataFrame({c: X[c], 'target': y})
            agg = df_temp.groupby(c)['target'].agg(['mean', 'count'])
            agg['enc'] = ((agg['count'] * agg['mean'] + self.smoothing * self.global_mean_) /
                          (agg['count'] + self.smoothing))
            self.mapping_[c] = agg['enc']
        return self
    def transform(self, X):
        X_copy = X.copy()
        for c in self.cols:
            X_copy[c] = X_copy[c].map(self.mapping_[c]).fillna(self.global_mean_)
        return X_copy

class MissingFlagImputer(BaseEstimator, TransformerMixin):
    """
    (FIXED) Imputes missing values and adds flags, ensuring consistent columns
    across different data splits to prevent errors.
    """
    def __init__(self, strategy='median'):
        self.strategy = strategy

    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=np.number).columns
        # Learn which columns have missing values in the training data
        self.cols_to_flag_ = [c for c in self.num_cols if X[c].isnull().any()]
        self.imputer_ = SimpleImputer(strategy=self.strategy)
        self.imputer_.fit(X[self.num_cols])
        return self

    def transform(self, X):
        X_copy = X.copy()
        # Create flags for all columns that were identified during fit
        for c in self.cols_to_flag_:
            X_copy[c + '_missing'] = X_copy[c].isnull().astype(int)

        # Impute the main numeric columns
        X_copy[self.num_cols] = self.imputer_.transform(X_copy[self.num_cols])
        return X_copy


class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_drop):
        self.cols_to_drop = cols_to_drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(columns=self.cols_to_drop, errors='ignore')

print("\nPART 1 Complete: All necessary libraries and custom classes are defined.")


# =============================================================================
# PART 2: DAILY AGGREGATION, EDA, AND VIF ANALYSIS
# -----------------------------------------------------------------------------
# This part creates the SIMPLE daily dataset for our baseline models.
# =============================================================================

print("\nPART 2: Aggregating data to daily level for baseline models...")
agg_dict = {
    'net_qty': 'sum', 'OrderQty': 'sum', 'CanceledQty': 'sum', 'MenuPrice': 'mean',
    'MenuSubsidy': 'mean', 'tavg_C': 'mean', 'prcp_mm': 'sum', 'rain_flag': 'max',
    'temp_dev': 'mean', 'is_holiday': 'max', 'is_weekend': 'max', 'MenuBase': 'nunique',
    'SchoolID': 'nunique', 'hist_cancel_rate': 'mean'
}
df_daily = df.groupby(['DateOfService', 'Site']).agg(agg_dict).reset_index()
df_daily['daily_cancel_pct'] = (df_daily['CanceledQty'] / df_daily['OrderQty']).fillna(0)
df_daily = df_daily.sort_values('DateOfService').set_index('DateOfService')
print("\nPART 2 Complete: Baseline daily data is ready.")


# =============================================================================
# PART 3: FEATURE ENGINEERING PIPELINE AND MODEL DEFINITION
# -----------------------------------------------------------------------------
# This part defines the pipeline for the baseline models.
# =============================================================================

print("\nPART 3: Defining the baseline feature engineering and regression pipeline...")

class DailyFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy['day_of_year'] = X_copy.index.dayofyear
        X_copy['weekday'] = X_copy.index.weekday
        X_copy['month'] = X_copy.index.month
        X_copy['year'] = X_copy.index.year
        X_copy['week_of_year'] = X_copy.index.isocalendar().week.astype(int)
        X_copy['quarter'] = X_copy.index.quarter
        for lag in [1, 2, 7, 14, 28]:
            X_copy[f'lag_{lag}'] = X_copy.groupby('Site')['net_qty'].shift(lag)
        X_copy['rolling_mean_7'] = X_copy.groupby('Site')['net_qty'].shift(1).rolling(window=7, min_periods=1).mean()
        X_copy['rolling_std_7'] = X_copy.groupby('Site')['net_qty'].shift(1).rolling(window=7, min_periods=1).std()
        return X_copy

def asymmetric_loss(y_true, y_pred, under_penalty=1.2):
    error = y_true - y_pred
    loss = np.mean(np.where(error < 0, -error * under_penalty, error))
    return loss
asymmetric_scorer = make_scorer(asymmetric_loss, greater_is_better=False)

features_to_exclude = ['net_qty', 'Site', 'OrderQty', 'CanceledQty', 'daily_cancel_pct']
regression_pipeline = Pipeline([
    ('daily_features', DailyFeatureEngineer()),
    ('imputer', MissingFlagImputer(strategy='median')),
    ('dropper', ColumnDropper(cols_to_drop=features_to_exclude)),
    ('regressor', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])
print("\nPART 3 Complete: Baseline pipeline is ready.")


# =============================================================================
# PART 4: BASELINE MODEL TRAINING & EVALUATION
# -----------------------------------------------------------------------------
# This part evaluates the simple baseline model.
# =============================================================================

print("\nPART 4: Starting BASELINE model training and evaluation...")
sites = df_daily['Site'].unique()
baseline_scores = {}
for site in sites:
    df_site = df_daily[df_daily['Site'] == site].copy()
    X, y = df_site, df_site['net_qty']
    if len(X) < 100: continue
    tscv = TimeSeriesSplit(n_splits=5)
    custom_scores = -cross_val_score(regression_pipeline, X, y, cv=tscv, scoring=asymmetric_scorer, n_jobs=-1)
    rmse_scores = -cross_val_score(regression_pipeline, X, y, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    baseline_scores[site] = {'Asymmetric Loss': custom_scores.mean(), 'RMSE': rmse_scores.mean()}
print("\nPART 4 Complete: Baseline model performance established.")


# =============================================================================
# PART 5: HYPERPARAMETER TUNING FOR BASELINE MODEL
# -----------------------------------------------------------------------------
# This part tunes the simple baseline model.
# =============================================================================

print("\nPART 5: Starting hyperparameter tuning for baseline model...")
param_dist_rf = {
    'regressor__n_estimators': [100, 200, 300, 500], 'regressor__max_features': ['sqrt', 'log2', 0.7, 0.8],
    'regressor__max_depth': [None, 10, 20, 30], 'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4], 'regressor__bootstrap': [True, False]
}
tuned_rf_models = {}
tuned_scores = {}
for site in sites:
    df_site = df_daily[df_daily['Site'] == site].copy()
    X, y = df_site, df_site['net_qty']
    if len(X) < 100: continue
    tscv = TimeSeriesSplit(n_splits=5)
    search = RandomizedSearchCV(
        estimator=regression_pipeline, param_distributions=param_dist_rf, n_iter=50,
        cv=tscv, scoring=asymmetric_scorer, n_jobs=-1, random_state=RANDOM_STATE, verbose=0
    )
    search.fit(X, y)
    tuned_rf_models[site] = search.best_estimator_
    tuned_rmse = -cross_val_score(search.best_estimator_, X, y, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1).mean()
    tuned_scores[site] = {'Asymmetric Loss': -search.best_score_, 'RMSE': tuned_rmse}
print("\nPART 5 Complete: Hyperparameter tuning for RandomForest finished.")


# =============================================================================
# PART 6: ADVANCED FEATURE ENGINEERING (MODIFIED)
# -----------------------------------------------------------------------------
# This step creates the advanced feature set by using the custom transformers
# on the original order-level data before aggregating.
# (FIX) Added calculation for 'daily_cancel_pct' to this dataframe.
# =============================================================================

print("\nPART 6: Performing advanced feature engineering...")
# Step 1: Create order-level features
hist_transformer = HistCancelRateTransformer(group_keys=('Site', 'MenuBase'), value_col='net_qty')
df_adv = hist_transformer.fit_transform(df)
cluster_transformer = ClusterTransformer(
    profile_feats=['hist_cancel_rate', 'rain_flag', 'temp_dev', 'MenuPrice', 'MenuSubsidy'],
    group_keys=('Site', 'MenuBase'), n_clusters=5
)
df_adv = cluster_transformer.fit_transform(df_adv)
# Step 2: Aggregate the newly enriched data to the daily level
adv_agg_dict = agg_dict.copy()
adv_agg_dict['cluster_id'] = pd.Series.mode
df_daily_adv = df_adv.groupby(['DateOfService', 'Site']).agg(adv_agg_dict).reset_index()
# (FIX) Calculate daily_cancel_pct for the advanced dataframe
df_daily_adv['daily_cancel_pct'] = (df_daily_adv['CanceledQty'] / df_daily_adv['OrderQty']).fillna(0)
df_daily_adv['cluster_id'] = df_daily_adv['cluster_id'].apply(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) > 0 else x).astype(int)
df_daily_adv = df_daily_adv.sort_values('DateOfService').set_index('DateOfService')
print("\nPART 6 Complete: Advanced feature set created.")


# =============================================================================
# PART 7: ADVANCED MODEL TRAINING (UNTUNED)
# -----------------------------------------------------------------------------
# This part evaluates the untuned advanced model to see its baseline performance.
# =============================================================================

print("\nPART 7: Building and evaluating an untuned advanced pipeline with LightGBM...")
features_to_exclude_adv = features_to_exclude + ['MenuBase']
advanced_pipeline_lgbm = Pipeline([
    ('daily_features', DailyFeatureEngineer()),
    ('imputer', MissingFlagImputer(strategy='median')),
    ('dropper', ColumnDropper(cols_to_drop=features_to_exclude_adv)),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])
advanced_lgbm_models = {}
advanced_scores = {}
for site in sites:
    df_site_adv = df_daily_adv[df_daily_adv['Site'] == site].copy()
    X, y = df_site_adv, df_site_adv['net_qty']
    if len(X) < 100: continue
    tscv = TimeSeriesSplit(n_splits=5)
    adv_custom_scores = -cross_val_score(advanced_pipeline_lgbm, X, y, cv=tscv, scoring=asymmetric_scorer, n_jobs=-1)
    adv_rmse_scores = -cross_val_score(advanced_pipeline_lgbm, X, y, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1)
    advanced_scores[site] = {'Asymmetric Loss': adv_custom_scores.mean(), 'RMSE': adv_rmse_scores.mean()}
    advanced_lgbm_models[site] = advanced_pipeline_lgbm.fit(X, y)
print("\nPART 7 Complete: Untuned advanced model performance established.")


# =============================================================================
# PART 8: INTERIM ANALYSIS AND VISUALIZATION (MODIFIED)
# -----------------------------------------------------------------------------
# This part generates a comprehensive analysis of the models built so far.
# The variable name has been fixed to prevent the NameError.
# =============================================================================

print("\nPART 8: Generating Interim Analysis and Visualizations...")
# --- Final Summary Table ---
print(f"\n{'='*30} INTERIM PERFORMANCE SUMMARY {'='*30}")
baseline_df = pd.DataFrame(baseline_scores).T.rename(columns=lambda c: f"Baseline RF {c}")
tuned_df = pd.DataFrame(tuned_scores).T.rename(columns=lambda c: f"Tuned RF {c}")
advanced_df = pd.DataFrame(advanced_scores).T.rename(columns=lambda c: f"Advanced LGBM {c}")
# (FIX) Renamed this variable for consistency
final_summary = pd.concat([baseline_df, tuned_df, advanced_df], axis=1)
print("Comparison of Model Performances:")
display(final_summary)


# =============================================================================
# PART 9 (FIX): ADVANCED CALENDAR FEATURES & NEW TARGET DEFINITION
# -----------------------------------------------------------------------------
# This new part creates more sophisticated holiday features and defines our
# new target variable: the daily cancellation rate, as suggested by the teacher.
# This approach avoids the leakage issues from hist_cancel_rate and SchoolID.
# =============================================================================

print("\nPART 9: Creating advanced calendar features and new cancellation rate target...")

# --- Advanced Calendar Features ---
# Use the advanced daily dataframe as our base
df_final = df_daily_adv.copy()

# Create a series of holiday dates for NRW, Germany
# Note: In a real project, this would use a library like `holidays` for accuracy
holiday_dates = pd.to_datetime(['2020-01-01', '2020-04-10', '2020-04-13', '2020-05-01', '2020-05-21', '2020-06-01', '2020-06-11', '2020-10-03', '2020-11-01', '2020-12-25', '2020-12-26',
                                '2021-01-01', '2021-04-02', '2021-04-05', '2021-05-01', '2021-05-13', '2021-05-24', '2021-06-03', '2021-10-03', '2021-11-01', '2021-12-25', '2021-12-26',
                                '2022-01-01', '2022-04-15', '2022-04-18', '2022-05-01', '2022-05-26', '2022-06-06', '2022-06-16', '2022-10-03', '2022-11-01', '2022-12-25', '2022-12-26',
                                '2023-01-01', '2023-04-07', '2023-04-10', '2023-05-01', '2023-05-18', '2023-05-29', '2023-06-08', '2023-10-03', '2023-11-01', '2023-12-25', '2023-12-26'])

# Calculate days to/from nearest holiday
date_index = df_final.index.to_series()
df_final['days_until_holiday'] = date_index.apply(lambda x: np.min(np.abs((holiday_dates - x).days)[holiday_dates > x]) if any(holiday_dates > x) else 0)
df_final['days_since_holiday'] = date_index.apply(lambda x: np.min(np.abs((x - holiday_dates).days)[holiday_dates < x]) if any(holiday_dates < x) else 0)

print("Advanced calendar features created:")
display(df_final[['is_holiday', 'days_until_holiday', 'days_since_holiday']].head())

# --- New Target Variable ---
# The target is now the daily cancellation percentage
df_final['target_cancel_pct'] = df_final['daily_cancel_pct']

print("\nPART 9 Complete: Advanced features and new target are ready.")


# =============================================================================
# PART 10: TWO-STAGE FORECASTING MODEL
# -----------------------------------------------------------------------------
# This part implements the final, robust two-stage model.
# Stage 1: Predict the cancellation rate.
# Stage 2: Predict the total order quantity.
# The two stages are combined for the final net_qty forecast.
# =============================================================================

print("\nPART 10: Training and evaluating the final two-stage forecasting model...")

# --- Define the Final Pipeline for Predicting Cancellation Rate ---
# We exclude features that could leak information about cancellations
rate_features_to_exclude = features_to_exclude + ['MenuBase', 'SchoolID', 'hist_cancel_rate', 'target_cancel_pct', 'daily_cancel_pct']
rate_pipeline = Pipeline([
    ('daily_features', DailyFeatureEngineer()),
    ('imputer', MissingFlagImputer(strategy='median')),
    ('dropper', ColumnDropper(cols_to_drop=rate_features_to_exclude)),
    ('regressor', LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])

final_two_stage_scores = {}
final_two_stage_models = {}

for site in sites:
    print(f"\n{'='*30} Evaluating Two-Stage Model for Site: {site} {'='*30}")

    df_site_final = df_final[df_final['Site'] == site].copy()

    if len(df_site_final) < 100:
        print(f"Skipping site {site} due to insufficient data.")
        continue

    tscv = TimeSeriesSplit(n_splits=5)

    # --- Custom Cross-Validation Loop for Two-Stage Model ---
    y_trues, y_preds = [], []
    for train_idx, test_idx in tscv.split(df_site_final):
        train_data, test_data = df_site_final.iloc[train_idx], df_site_final.iloc[test_idx]

        # --- Stage 1: Train and Predict Cancellation Rate ---
        X_rate_train, y_rate_train = train_data, train_data['target_cancel_pct']
        rate_pipeline.fit(X_rate_train, y_rate_train)
        predicted_cancel_pct = rate_pipeline.predict(test_data)
        # Ensure predictions are between 0 and 1
        predicted_cancel_pct = np.clip(predicted_cancel_pct, 0, 1)

        # --- Stage 2: Predict Order Quantity (using a simple baseline) ---
        # Baseline: OrderQty will be the same as last week's OrderQty
        train_data['lag_7_order_qty'] = train_data['OrderQty'].shift(7)
        last_known_lag = train_data['lag_7_order_qty'].dropna().iloc[-1]
        predicted_order_qty = pd.Series(last_known_lag, index=test_data.index) # Simple propagation

        # --- Combine for Final Forecast ---
        final_predictions = predicted_order_qty * (1 - predicted_cancel_pct)

        y_trues.append(test_data['net_qty'])
        y_preds.append(final_predictions)

    # Concatenate results from all folds
    y_true_all = pd.concat(y_trues)
    y_pred_all = pd.concat(y_preds)

    # Calculate final scores on the original scale
    final_rmse = np.sqrt(np.mean((y_true_all - y_pred_all)**2))
    final_asymmetric_loss = asymmetric_loss(y_true_all, y_pred_all)

    final_two_stage_scores[site] = {'Asymmetric Loss': final_asymmetric_loss, 'RMSE': final_rmse}
    print(f"    - Final Two-Stage Model Asymmetric Loss: {final_asymmetric_loss:.2f}")
    print(f"    - Final Two-Stage Model RMSE: {final_rmse:.2f} meals")

    # Fit the final rate model on all data for SHAP analysis
    final_two_stage_models[site] = rate_pipeline.fit(df_site_final, df_site_final['target_cancel_pct'])


# --- Final Summary Table with All Models ---
print(f"\n\n{'='*30} OVERALL PERFORMANCE SUMMARY {'='*30}")
final_model_df = pd.DataFrame(final_two_stage_scores).T.rename(columns=lambda c: f"Final Two-Stage LGBM {c}")
overall_summary = pd.concat([final_summary, final_model_df], axis=1)
print("Comparison of All Model Performances:")
display(overall_summary)

# --- Detailed Visualization of the FINAL Two-Stage Model ---
for site, best_pipeline in final_two_stage_models.items():
    print(f"\n{'='*30} SHAP Analysis for FINAL Two-Stage Model at Site: {site} {'='*30}")

    df_site = df_final[df_final['Site'] == site].copy()
    X, y = df_site, df_site['target_cancel_pct']

    # Transform data for SHAP
    X_featured = best_pipeline.named_steps['daily_features'].transform(X)
    X_imputed = best_pipeline.named_steps['imputer'].transform(X_featured)
    X_transformed = best_pipeline.named_steps['dropper'].transform(X_imputed)

    regressor = best_pipeline.named_steps['regressor']
    explainer = shap.TreeExplainer(regressor)
    shap_values = explainer.shap_values(X_transformed)

    plt.title(f'SHAP Feature Importance for FINAL Two-Stage Model at Site: {site}', fontsize=16, weight='bold')
    shap.summary_plot(shap_values, X_transformed, plot_type="bar", show=False)
    plt.show()

print("\nSCRIPT COMPLETE: Final two-stage model has been evaluated.")


/Users/shayan/Desktop/IDS2/Stattkueche/venv_arm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PART 1: Loading and preparing data...


KeyboardInterrupt: 

In [4]:

# =============================================================================
# PART 11: IN-DEPTH FINAL MODEL ANALYSIS (NEW)
# -----------------------------------------------------------------------------
# This new final part provides a deep dive into the performance and logic
# of the best model (the Two-Stage Model from Part 10), with detailed
# visualizations and metrics as requested.
# =============================================================================

print(f"\n\n{'='*30} PART 11: IN-DEPTH ANALYSIS OF THE FINAL TWO-STAGE MODEL {'='*30}")

# --- Final Performance Table ---
final_model_df = pd.DataFrame(final_two_stage_scores).T.rename(columns=lambda c: f"Final Two-Stage LGBM {c}")
overall_summary = pd.concat([final_summary, final_model_df], axis=1)
print("--- Overall Performance Summary ---")
display(overall_summary)

# --- Detailed Visualization Loop ---
for site, final_model_pipeline in final_two_stage_models.items():
    print(f"\n{'='*30} Detailed Analysis for Site: {site} {'='*30}")

    df_site = df_final[df_final['Site'] == site].copy()
    X, y = df_site, df_site['target_cancel_pct']

    # --- Prepare Data for SHAP and Other Plots ---
    X_featured = final_model_pipeline.named_steps['daily_features'].transform(X)
    X_imputed = final_model_pipeline.named_steps['imputer'].transform(X_featured)
    X_transformed = final_model_pipeline.named_steps['dropper'].transform(X_imputed)
    regressor = final_model_pipeline.named_steps['regressor']

    # --- 1. SHAP Feature Importance ---
    print("\n--> 1. SHAP Feature Importance (What the model thinks is important)")
    explainer = shap.TreeExplainer(regressor)
    shap_values = explainer.shap_values(X_transformed)
    plt.title(f'SHAP Feature Importance for Final Two-Stage Model at Site: {site}', fontsize=16, weight='bold')
    shap.summary_plot(shap_values, X_transformed, plot_type="bar", show=False)
    plt.show()

    # --- 2. SHAP Dependence Plots for Top Features ---
    print("\n--> 2. SHAP Dependence Plots (How top features affect predictions)")
    top_features = X_transformed.columns[np.argsort(np.abs(shap_values).mean(0))][::-1]
    for feature in top_features[:3]: # Plot top 3 features
        shap.dependence_plot(feature, shap_values, X_transformed, interaction_index=None, show=False)
        plt.title(f'SHAP Dependence Plot for "{feature}" at Site: {site}', fontsize=14)
        plt.show()

    # --- 3. Error Analysis ---
    # Re-run the CV loop to get the final predictions and errors for plotting
    tscv = TimeSeriesSplit(n_splits=5)
    y_trues, y_preds = [], []
    for train_idx, test_idx in tscv.split(df_site):
        train_data, test_data = df_site.iloc[train_idx], df_site.iloc[test_idx]
        X_rate_train, y_rate_train = train_data, train_data['target_cancel_pct']
        final_model_pipeline.fit(X_rate_train, y_rate_train)
        predicted_cancel_pct = np.clip(final_model_pipeline.predict(test_data), 0, 1)
        train_data['lag_7_order_qty'] = train_data['OrderQty'].shift(7)
        last_known_lag = train_data['lag_7_order_qty'].dropna().iloc[-1]
        predicted_order_qty = pd.Series(last_known_lag, index=test_data.index)
        final_predictions = predicted_order_qty * (1 - predicted_cancel_pct)
        y_trues.append(test_data['net_qty'])
        y_preds.append(final_predictions)

    y_true_all = pd.concat(y_trues)
    y_pred_all = pd.concat(y_preds)
    errors = y_true_all - y_pred_all

    print("\n--> 3. Prediction Error Analysis")
    print("Descriptive Statistics of Prediction Errors:")
    display(errors.describe())

    # Plot: Error Time Series
    plt.figure(figsize=(18, 6))
    errors.plot(style='.', alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.title(f'Prediction Errors Over Time for Site: {site}', fontsize=16, weight='bold')
    plt.ylabel('Error (Actual - Predicted)')
    plt.show()

    # Plot: Actual vs. Predicted Scatter Plot
    plt.figure(figsize=(8, 8))
    plt.scatter(y_true_all, y_pred_all, alpha=0.5)
    plt.plot([y_true_all.min(), y_true_all.max()], [y_true_all.min(), y_true_all.max()], '--', color='red', lw=2)
    plt.title(f'Actual vs. Predicted Net Quantity for Site: {site}', fontsize=16, weight='bold')
    plt.xlabel('Actual Net Quantity')
    plt.ylabel('Predicted Net Quantity')
    plt.show()


print("\nSCRIPT COMPLETE: Final in-depth analysis has been generated.")




============================== PART 11: IN-DEPTH ANALYSIS OF THE FINAL TWO-STAGE MODEL ==============================


NameError: name 'pd' is not defined